# Dog breed classification using Pytorch Estimators on Azure Machine Learning service

Have you ever seen a dog and not been able to tell the breed? Some dogs look so similar, that it can be nearly impossible to tell. For instance these are a few breeds that are difficult to tell apart:

#### Alaskan Malamutes vs Siberian Huskies
![Image of Alaskan Malamute vs Siberian Husky](http://cdn.akc.org/content/article-body-image/malamutehusky.jpg)

#### Whippet vs Italian Greyhound 
![Image of Whippet vs Italian Greyhound](http://cdn.akc.org/content/article-body-image/whippetitalian.jpg)

There are sites like http://what-dog.net, which use Microsoft Cognitive Services to be able to make this easier. 

In this tutorial, you will learn how to train a Pytorch image classification model using transfer learning with the Azure Machine Learning service. The Azure Machine Learning python SDK's [PyTorch estimator](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-pytorch) enables you to easily submit PyTorch training jobs for both single-node and distributed runs on Azure compute. The model is trained to classify dog breeds using the [Stanford Dog dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) and it is based on a pretrained ResNet18 model. This ResNet18 model has been built using images and annotation from ImageNet. The Stanford Dog dataset contains 120 classes (i.e. dog breeds), to save time however, for most of the tutorial, we will only use a subset of this dataset which includes only 10 dog breeds.

## Setup


### Conda/Miniconda
[Download](https://conda.io/miniconda.html) and install Miniconda. Select the Python 3.7 version or later. Don't select the Python 2.x version.

### Create Conda Environment

Start with the conda environment definition file:

```
name: learnai-hackathon

dependencies:
  - python=3.6

  - pip:
    - azureml-sdk[notebooks,contrib]
    - https://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-win_amd64.whl
    - torchvision

```

*Note*: The above instructions are for windows. If you are using a different OS, you may need to replace the last two lines above with something else, following [these](https://pytorch.org/) (section `Quick Start Locally`) instructions.

Add the following *conda* dependencies for advanced jupyter notebook features (such as widgets): `ipywidgets`, `nb_conda`.

Add the following *pip* dependencies for deep learning: `scikit-image`.

Once you are done editing the file, create the conda environment.


### Install jupyter notebook extensions for widgetes

Run the following installation steps on the command line (e.g. Anaconda Shell)

Install the Python SDK:  make sure to install notebook, and contrib
``` 
jupyter nbextension install --py --user azureml.widgets
jupyter nbextension enable azureml.widgets --user --py
```


### Start Jupyter Notebook server
Change directory into the folder of our repository, activate the above conda environment, and type `jupyter notebook` to start the jupyter server. 


### Test whether SDK was installed correctly in your conda environment 

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.8


## Train an image classification model?
Training machine learning models, particularly deep neural networks, is often a time- and compute-intensive task. Once you've finished writing your training script and running on a small subset of data on your local machine, you will likely want to scale up your workload.

To facilitate training, the Azure Machine Learning Python SDK provides a high-level abstraction, the estimator class, which allows users to easily train their models in the Azure ecosystem. You can create and use an Estimator object to submit any training code you want to run on remote compute, whether it's a single-node run or distributed training across a GPU cluster. For PyTorch and TensorFlow jobs, Azure Machine Learning also provides respective custom PyTorch and TensorFlow estimators to simplify using these frameworks.

### Steps to train with a Pytorch Estimator:
In this tutorial, we will:
- Connect to an Azure Machine Learning service Workspace 
- Create a remote compute target
- Download your training data (Optional)
- Create your training script
- Create an Estimator object
- Submit your training job

## Create workspace

In the next step, you will create your own Workspace to use in this tutorial.

**You will be asked to login during this step. Please use your Microsoft AAD credentials.**

In [ ]:
## need to update the following 2 values based on the sub we are using
subscription_id = '...'                           # <<please retrieve from URL at above paragraph>>
workspace_name = '...'                            # <<a Unique Name -- use your alias>>
location = '...'
resource_group = '...'

Next, create a workspace, as we have before.

In [ ]:
from azureml.core import Workspace


ws = '...'

ws.write_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

This will take a few minutes, so let's talk about what a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) is while it is being created.
![](aml-workspace.png)


## Create a remote compute target
For this tutorial, we will create an AML Compute cluster as we have before, using a `Standard_NC6` VM, which include P100 GPUs for deep learning, as the [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) to execute your training script on. 

**Creation of the cluster takes approximately 5 minutes** 


In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

# choose a name for your cluster
cluster_name = "..."

try:
    # todo
    print('Found existing compute target.')
except KeyError:
    print('Creating a new compute target...')
    # todo

In [ ]:
# you can ask for the provisioning state to see whether the cluster is up already of if there were errors
compute_target.status.serialize()

## Attach the blobstore with the training data to the workspace
While the cluster is still creating, let's attach some data to our workspace.

The dataset we will use consists of ~150 images per class. Some breeds have more, while others have less. Each class has about 100 training images each for dog breeds, with ~50 validation images for each class. 

Two datasets are available.  The first smaller dataset with only 10 dog breeds we will use for developing our approach. The second full dataset contains 120 dog breeds.  This dataset is very large, so let's ignore it for now. 

(optional) Download these two datasets as zip files and look at a couple of them to become familiar with what your model will be asked to accomplish.
- [10 breeds](https://github.com/heatherbshapiro/pycon-canada/blob/master/breeds-10.zip?raw=true)
- [120 breeds](https://github.com/heatherbshapiro/pycon-canada/blob/master/breeds.zip?raw=true)

To make the data accessible for remote training, you will need to keep the data in the cloud. AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data, and interact with it from your remote compute targets. It is an abstraction over Azure Storage. The datastore can reference either an Azure Blob container or Azure file share as the underlying storage. 

We already put these data into blob for you. Just ask for us for the account name to fill in below!

In [ ]:
from azureml.core import Datastore
ds = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name='breeds', 
                                             container_name='dogbreeds',
                                             account_name='...', 
                                             account_key='uzFqBxsdDHPlxrM1hc4vdFpJrkEU/E7af+dybu4iVLm4FQEMIhscxOR+/tWXAIr23qLRaEpkVSAIELTCR/Rnjw==')

### Upload your train and test data to blob storage

We already uploaded the data, so you don't need to do this again. We are leaving this here, in case you want to use a different dataset sometime.

In [ ]:
# ds.upload("breeds-10")
# ds.upload("breeds")

Now let's get a reference to the path on the datastore with the training data. We can do so using the `path` method. In the next section, we can then pass this reference to our training script's `--data_dir` argument. We will start with the 10 classes dataset.

In [ ]:
path_on_datastore = 'breeds-10'
ds_data = ds.path(path_on_datastore)
print("Data store reference: %s " % ds_data)

## Download the Data

If you are interested in downloading the data locally, you can run `ds.download(".", 'breeds-10')`. This might take several minutes and not necessary whatsover for now.

In [ ]:
ds.download('.', 'breeds-10', show_progress=True)

### Prepare training script
Now you will need to create your training script. In this tutorial, the training script is already provided for you at `pytorch_train_orig.py`. But you still need to make modifications to it, so that it properly logs results in Azure. We recommend that you leave the above `_orig.py` script in place, and make a copy of it at `pytorch_trian.py`.

However, if you would like to use AML's [tracking and metrics](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#metrics) capabilities, you will have to add a small amount of AML code inside your training script. 

In `pytorch_train.py`, we will log some metrics to our AML run. To do so, we will access the AML run object within the script:

```Python
from azureml.core.run import Run
run = Run.get_context()
```

Further within the method `fixed_feature_model` in `pytorch_train.py`, we log the learning rate and momentum parameters, the number of classes in the model:

```Python
    run.log('lr', np.float(learning_rate))
    run.log('momentum', np.float(momentum))
    run.log('num_classes', num_classes)
```

This might be a good opportunity to investigate the order of execution of the various methods in this script.

We also need to log the best validation accuracy (`best_acc`) the model achieves in `train_model`:
```Python
            # log the best val accuracy to AML run
            if phase == 'val': 
                run.log('best_val_acc', np.float(best_acc))
```

If you downloaded the data, you can start to train the model locally (note that it will take long if you don't have a GPU -- 21 min. on a Core i7 CPU).

In [ ]:
!mkdir outputs
!python pytorch_train.py --data_dir breeds-10 --num_epochs 1 --output_dir outputs 

## Train model on the remote compute
Now that you have your data and training script prepared, you are ready to train on your remote compute cluster. You can take advantage of Azure compute to leverage GPUs to cut down your training time. 

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this transfer learning PyTorch tutorial. 

In [ ]:
from #todo import #todo

experiment_name = 'pytorch-dogs'
#todo 

### Create a PyTorch estimator
The AML SDK's PyTorch estimator enables you to easily submit PyTorch training jobs for both single-node and distributed runs. For more information on the PyTorch estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-pytorch). The following code will define a single-node PyTorch job.

There are two things you need to do here. 
1. Look into the file `pytorch_train.py` and determine which `script_params` you need to specify.
1. Specify the compute_target on which you want to train your model.

The `script_params` parameter is a dictionary containing the command-line arguments to your training script `entry_script`. Please note the following:
- We pass our training data reference `ds_data` to our script's `--data_dir` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the training data `breeds` on our datastore.
- We specified the output directory as `./outputs`. The `outputs` directory is specially treated by AML in that all the content in this directory gets uploaded to your workspace as part of your run history. The files written to this directory are therefore accessible even once your remote run is over. In this tutorial, we will save our trained model to this output directory.
- We recommend that you train for 10 epochs.

In [ ]:
from azureml.train.dnn import PyTorch

script_params = {
    # todo, looks at the file pytorch_train.py, and determine 
    # which script_params you need to specify.
}

estimator10 = PyTorch(source_directory='.', 
                    script_params=script_params,
                    compute_target=#todo, 
                    entry_script='pytorch_train.py',
                    pip_packages=['tensorboardX'],
                    use_gpu=True)


To leverage the Azure VM's GPU for training, we set `use_gpu=True`.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [ ]:
# todo

### Monitor your run
You can monitor the progress of the run with a Jupyter widget (class `RunDetails` of module `azureml.widgets`). Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
# todo

### What happens during a run?
If you are running this for the first time, the compute target will need to pull the docker image, which will take about 2 minutes. This gives us the time to go over how a **Run** is executed in Azure Machine Learning. 

Note: had we not created the workspace with an existing ACR, we would have also had to wait for the image creation to be performed -- that takes and extra 10-20 minutes for big GPU images like this one. This is a one-time cost for a given python configuration, and subsequent runs will then be faster. We are working on ways to make this image creation faster.

![](aml-run.png)

### Using Tensorboard
Tensorboard is a popular Deep Learning Training visualization tool. It is part of TensorFlow framework, but can be used from PyTorch as well by using TensorboadX python package. TensorboardX allows PyTorch to write metrics and log information in Tensorboard format.

In `pytorch_train.py`, we will log metrics to "magical" `logs` directory. The content of this special directory is automatically streamed by Azue ML service. The logging into Tensorboard event format is performed by SummaryWriter object im the `main` method:

```Python
    from tensorboardX import SummaryWriter
    writer = SummaryWriter(f'./logs/{run.id}')
```

Within the script (method `train_model`) we write more detailed mini-batch loss and accuracy:
```Python
                writer.add_scalar(f'{phase}/Loss', loss.item(), niter)
                writer.add_scalar(f'{phase}/Accuracy', (corrects / inputs.size(0)).item(), niter)
```

We also log the epoch level accuracy to Tensorboard:
```Python
            writer.add_scalar(f'{phase}/Epoch_accuracy', epoch_acc, (epoch+1) * len(dataloaders[phase]))
```

Finally, to ensure that TensorboardX python package is installed in Python environment during the training run, we add it using pip_packages parameter of the Estimator object:
```Python
estimator = PyTorch(...
                    pip_packages=['tensorboardX']
                    ...)
```

### Installing and launching Tensorboard
Azure ML SDK provides built-in integration with Tensorboard in package `azureml-contrib-tensorboard`, installed as part of the contrib extras package in prerequisites. In addition, you will need to pip install tensorboard, which we also did as part of the prerequisites.

Edit your `environment.yml` file, to also include tensorflow (`tensorflow` and `tensorboardX`), and update your conda environment (`conda env update -f environment.yml`).

While the run is in progress (or after it has completed), we just need to start Tensorboard with the run as its target, and it will begin streaming logs.

In [ ]:
from azureml.contrib.tensorboard import Tensorboard

print("Click on the http link to access your Tensorboard!")

# The Tensorboard constructor takes an array of runs, so be sure and pass it in as a single-element array here
tb = Tensorboard([run10])

# If successful, start() returns a string with the URI of the instance.
tb.start()

### Stop Tensorboard

When you're done, make sure to call the `stop()` method of the Tensorboard object, or it will stay running even after your job completes.

In [ ]:
tb.stop()

### (optional) Try a different optimizer

The script uses a standard stochastic gradient (SGD) optimizer for optimizing weights in the neural network.  This might now be the best option.  Go to the torch [documentation](https://pytorch.org/docs/stable/optim.html) and see what other optimizers are available.  For example, you could try the `Adam` optimizer to see whether your model learns faster or achieves better validation accuracy with that.

### (optional) Try a different architecture

If you take a close look at pytorch_train.py, you will notice that it uses a pretrained ResNet18 architecture.  You could try a different one, and see whether that improves your performance.

### (optional) Try training the model from scratch

Whatever architecture you decide on, consider training it from scratch.

### (optional) Try different kinds of pre-processing

Check out the load_data method in `pytorch_train.py`. You will noticed that there are a couple of sequential preprocessing steps. Think about playing with those, maybe there are better ways to preprocess your data.


### Fine Tuning

Now that the setup is working, we can go to the full dataset with 120 classes. We just need to point to a different path on the datastore. 

In [ ]:
full_dataset = ds.path('breeds')
print(full_dataset)

Update the script_params.  We recommend training for 25 `epochs`. Make sure to set the `mode` to `fine_tune` ([see fast.ai wiki](http://wiki.fast.ai/index.php/Fine_tuning)).



In [ ]:
## AML Compute
from azureml.train.dnn import PyTorch

script_params = {
    # ... 
}

estimator120 = PyTorch(source_directory='.', 
                        script_params=script_params,
                        compute_target=compute_target, 
                        entry_script='pytorch_train.py',
                        pip_packages=['tensorboardX'],
                        node_count=1,
                        use_gpu=True)

run120 = experiment.submit(estimator120)

### (optional) Distributed Training with Horovod

But now training takes very long (1.5 hours), so let's see if we can run this job on multiple GPUs to cut down on training time.

In [ ]:
# first let's cancel the above job
run120.cancel()

Running the model on multiple nodes is simple (in this case using Horovod MPI-based algorithm running on 4 nodes)

In [ ]:
## AML Compute
from azureml.train.dnn import PyTorch

script_params = {
    # you can use the same params as above
}

estimator120 = PyTorch(source_directory='.', 
                        script_params=script_params,
                        compute_target=compute_target, 
                        pip_packages=['tensorboardX'],
                        entry_script='pytorch_train_horovod.py',
                        node_count=4,
                        distributed_backend='mpi',
                        use_gpu=True)

run120 = experiment.submit(estimator120)

In [ ]:
# use the azureml.widgets to monitor the run

In [ ]:
from azureml.contrib.tensorboard import Tensorboard

# The Tensorboard constructor takes an array of runs, so be sure and pass it in as a single-element array here
tb = Tensorboard([run120])

# If successful, start() returns a string with the URI of the instance.
tb.start()

In [ ]:
tb.stop()

Training on 4 nodes completes in about 10 minutes and achieves 76% accuracy, which is similar to accuracy produced by single node training. This is great improvement of training time.

### Hyperparameter Tuning
 Now that you have trained an initial model, you can tune the hyperparameters of this model to optimize model performance. Azure ML allows you to automate this tuning, in an efficient manner via early termination of poorly performing runs.

You can configure your Hyperparamter Tuning experiment by specifying the following info -
- Define the hyparparameter space - specify ranges, distribution and sampling
- Early Termination policy
- Optimization metric
- Resource / Compute budget
- Desired concurrency


In [1]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive import *

ps = RandomParameterSampling(
    {
        # todo 
        # todo 
    }
)

policy = BanditPolicy() #todo


hdc = HyperDriveRunConfig(estimator=estimator10, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name=#todo,
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=50,
                          max_concurrent_runs=4)

hd_run = experiment.submit(hdc)
RunDetails(hd_run).show()

NameError: name 'estimator10' is not defined

While the jobs is running, take a look at the [Hyperdrive documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py) to see what other options Hyperdrive offers.

In [ ]:
# at any time, you can pull out the metrics generated so far from the runs
hd_run.get_metrics()

### Report your results

What is the best score if have a achieved and want to share with the group? 
What do you think were the most important changes that allowed you to get there?

## Inferencing
### Create scoring script

First, we will create a scoring script that will be invoked by the web service call. Note that the scoring script must have two required functions:
* `init()`: In this function, you typically load the model into a `global` object. This function is executed only once when the Docker container is started. 
* `run(input_data)`: In this function, the model is used to predict a value based on the input data. The input and output typically use JSON as serialization and deserialization format, but you are not limited to that.

Refer to the scoring script `pytorch_score.py` for this tutorial. Our web service will use this file to predict the dog breed based on a 120 class model trained earlier, located in the folder `model`. We will test the scoring file locally first before you go and deploy the web service.

1. Import the scoring script, so that init() and run() are accessible

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pytorch_score

2. Call the init function -- this will load the model and the class_names from the model directory

In [ ]:
pytorch_score.init()

3. Add some helper functions:

In [ ]:
import os, json, base64
from io import BytesIO
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
import urllib.request
import io, requests

##Get random dog
def get_random_dog():
    r = requests.get(url ="https://dog.ceo/api/breeds/image/random")
    URL= r.json()['message']
    return URL

def imgToBase64(img):
    """Convert pillow image to base64-encoded image"""
    imgio = BytesIO()
    img.save(imgio, 'JPEG')
    img_str = base64.b64encode(imgio.getvalue())
    return img_str.decode('utf-8')

4. Find an image of a dog and call the run() function

In [ ]:
##Get Random Dog Image
URL = get_random_dog()

with urllib.request.urlopen(URL) as url:
    test_img = io.BytesIO(url.read())


plt.imshow(Image.open(test_img))

base64Img = imgToBase64(Image.open(test_img))

result = pytorch_score.run(input_data=json.dumps({'data': base64Img}))
print(URL)
print(json.loads(result))

### Register the model
Once the run completes, we can register the model that was created.

**Please use a unique name for the model**

In [ ]:
from azureml.core.model import Model
model = Model.register(ws, model_name='model', model_path = 'model', description='120 Dogbreeds')
print(model.name, model.id, model.version, sep = '\t')

## (optional) Deploy model as web service
Once you have your trained model, you can deploy the model on Azure. You can deploy your trained model as a web service on Azure Container Instances (ACI), Azure Kubernetes Service (AKS), IoT edge device, or field programmable gate arrays (FPGAs)

ACI is generally cheaper than AKS and can be set up in 4-6 lines of code. ACI is the perfect option for testing deployments. Later, when you're ready to use your models and web services for high-scale, production usage, you can deploy them to AKS.


In this tutorial, we will deploy the model as a web service in [Azure Container Instances](https://docs.microsoft.com/en-us/azure/container-instances/) (ACI). 


For more information on deploying models using Azure ML, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-deploy-and-where).

### Create environment file
Then, we will need to create an environment file (`myenv.yml`) that specifies all of the scoring script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image by AML. In this case, we need to specify `torch`, `torchvision`, `pillow`, and `azureml-sdk`.

In [ ]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - torch
    - torchvision
    - pillow
    - azureml-core

### Configure the container image
Now configure the Docker image that you will use to build your ACI container.

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script='pytorch_score.py', 
                                                  runtime='python', 
                                                  conda_file='myenv.yml',
                                                  description='Image with dog breed model')

### Configure the ACI container
We are almost ready to deploy. Create a deployment configuration file to specify the number of CPUs and gigabytes of RAM needed for your ACI container. While it depends on your model, the default of `1` core and `1` gigabyte of RAM is usually sufficient for many models.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={'data': 'dog_breeds',  'method':'transfer learning', 'framework':'pytorch'},
                                               description='Classify dog breeds using transfer learning with PyTorch')

### Deploy the registered model
Finally, let's deploy a web service from our registered model. First, retrieve the model from your workspace.

In [ ]:
from azureml.core.model import Model

model = ws.models['model']

Then, deploy the web service using the ACI config and image config files created in the previous steps. We pass the `model` object in a list to the `models` parameter. If you would like to deploy more than one registered model, append the additional models to this list.

**Please use a unique service name**

In [ ]:
%%time
from azureml.core.webservice import Webservice

service_name = 'dog120'
service = Webservice.deploy_from_model(workspace=ws,
                                       name=service_name,
                                       models=[model],
                                       image_config=image_config,
                                       deployment_config=aciconfig,)

service.wait_for_deployment(show_output=True)
print(service.state)

If your deployment fails for any reason and you need to redeploy, make sure to delete the service before you do so: `service.delete()`

**Tip: If something goes wrong with the deployment, the first thing to look at is the logs from the service by running the following command:**

In [ ]:
service.get_logs()


Get the web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(service.scoring_uri)

### Test the web service
Finally, let's test our deployed web service. We will send the data as a JSON string to the web service hosted in ACI and use the SDK's `run` API to invoke the service. Here we will take an arbitrary image from online to predict on. This is the same as above, but now we are testing on our own trained model. You can use any dog image, but please remember we only trained on 10 classes.

In [ ]:
##Get Random Dog Image
URL = get_random_dog()

with urllib.request.urlopen(URL) as url:
    test_img = io.BytesIO(url.read())

plt.imshow(Image.open(test_img))

with urllib.request.urlopen(URL) as url:
    test_img = io.BytesIO(url.read())

plt.imshow(Image.open(test_img))
print(URL)

In [ ]:
def imgToBase64(img):
    """Convert pillow image to base64-encoded image"""
    imgio = BytesIO()
    img.save(imgio, 'JPEG')
    img_str = base64.b64encode(imgio.getvalue())
    return img_str.decode('utf-8')

base64Img = imgToBase64(Image.open(test_img))

result = service.run(input_data=json.dumps({'data': base64Img}))
print(json.loads(result))

### Delete web service
Once you no longer need the web service, you should delete it.

In [2]:
# service.delete()